In [0]:
from pyspark.sql.functions import col

# -----------------------
# LOAD GOLD TABLES
# -----------------------

cliff = spark.table("workspace.gold.cliff_proximity_metrics").alias("c")
mc = spark.table("workspace.gold.monte_carlo_risk_metrics").alias("m")

# -----------------------
# JOIN DETERMINISTIC + STOCHASTIC
# -----------------------

profile = (
    cliff
    .join(
        mc,
        (col("c.user_id") == col("m.user_id")) &
        (col("c.plan_id") == col("m.plan_id")),
        how="inner"
    )
)

# -----------------------
# EXPECTED ANNUAL TOTAL COST
# -----------------------

profile = profile.withColumn(
    "expected_annual_total_cost",
    (col("c.net_premium") * 12) + col("m.mean_oop")
)

# -----------------------
# FINAL SELECT
# -----------------------

final_profile = profile.select(
    col("c.user_id").alias("user_id"),
    col("c.plan_id").alias("plan_id"),
    col("c.net_premium"),
    col("c.fragility_slope"),
    col("c.elasticity_ratio"),
    col("m.breach_probability"),
    col("m.mean_oop"),
    col("m.p90_exposure"),
    col("expected_annual_total_cost")
)

spark.sql("DROP TABLE IF EXISTS workspace.gold.plan_fragility_profile")

final_profile.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("workspace.gold.plan_fragility_profile")

In [0]:
import json
from pyspark.sql.functions import col

profile_df = spark.table("workspace.gold.plan_fragility_profile")

# Convert entire table to Pandas
profile_pd = profile_df.toPandas()

# Save locally inside driver memory
file_path = "/tmp/fragility_profiles.json"

profile_pd.to_json(file_path, orient="records", indent=2)

file_path

In [0]:
profile_df = spark.table("workspace.gold.plan_fragility_profile")

json_str = profile_df.toPandas().to_json(orient="records", indent=2)

displayHTML(f"""
<textarea style="width:100%; height:500px;">
{json_str}
</textarea>
""")